In [ ]:
# FIX: Downgrade PyTorch to a YOLOv8-compatible version

!pip uninstall -y torch torchvision torchaudio
!pip install torch==2.5.1 torchvision==0.20.1 torchaudio==2.5.1


In [ ]:
# Block 1: Mount Google Drive and create project structure

from google.colab import drive
import os

drive.mount('/content/drive')

PROJECT_ROOT = "/content/drive/MyDrive/ChildSafetyProject1"
RAW_IMAGES_DIR = os.path.join(PROJECT_ROOT, "raw_images")
YOLO_DATASET_DIR = os.path.join(PROJECT_ROOT, "yolo_dataset")

os.makedirs(RAW_IMAGES_DIR, exist_ok=True)
os.makedirs(YOLO_DATASET_DIR, exist_ok=True)

print("Project folders ready")


In [ ]:
# Block 3: Generate synthetic images with Stable Diffusion

import torch
from diffusers import StableDiffusionPipeline
import os

pipe = StableDiffusionPipeline.from_pretrained(
    "runwayml/stable-diffusion-v1-5",
    torch_dtype=torch.float16
)
pipe = pipe.to("cuda")

danger_prompts = {
    "scissors": "sharp metal scissors on a child's bedroom floor, toys around, realistic photo",
    "battery": "small button battery on a nursery carpet, close-up, realistic",
    "detergent": "colorful detergent bottle on a low shelf in a playroom, realistic",
    "wires": "damaged electrical wire with exposed copper on bedroom floor, realistic"
}

IMAGES_PER_CLASS = 10

for label, prompt in danger_prompts.items():
    for i in range(IMAGES_PER_CLASS):
        image = pipe(prompt, guidance_scale=7.5, num_inference_steps=30).images[0]
        image.save(os.path.join(RAW_IMAGES_DIR, f"{label}_{i}.jpg"))

print("Image generation completed")

In [ ]:
import os
!pip install autodistill-grounding-dino==0.1.3 roboflow --quiet

from autodistill_grounding_dino import GroundingDINO
from autodistill.detection import CaptionOntology

ontology = CaptionOntology({
    "scissors": "Sharp Object",
    "battery": "Choking Hazard",
    "detergent bottle": "Chemical Danger",
    "exposed electrical wire": "Electrical Hazard"
})

model = GroundingDINO(ontology=ontology)

dataset = model.label(
    input_folder=RAW_IMAGES_DIR,
    output_folder=YOLO_DATASET_DIR,
    extension=".jpg"
)

print("Automatic labeling completed")

In [ ]:
# Block 5: Train/Validation split

import os
import shutil
import random

random.seed(42)

images_dir = os.path.join(YOLO_DATASET_DIR, "images")
labels_dir = os.path.join(YOLO_DATASET_DIR, "labels")

for split in ["train", "val"]:
    os.makedirs(os.path.join(images_dir, split), exist_ok=True)
    os.makedirs(os.path.join(labels_dir, split), exist_ok=True)

all_images = [f for f in os.listdir(images_dir) if f.endswith(".jpg")]
random.shuffle(all_images)

split_idx = int(0.8 * len(all_images))
train_imgs = all_images[:split_idx]
val_imgs = all_images[split_idx:]

def move_files(img_list, split):
    for img in img_list:
        shutil.move(os.path.join(images_dir, img), os.path.join(images_dir, split, img))
        lbl = img.replace(".jpg", ".txt")
        shutil.move(os.path.join(labels_dir, lbl), os.path.join(labels_dir, split, lbl))

move_files(train_imgs, "train")
move_files(val_imgs, "val")

print("Train/Val split done")

In [ ]:
# Block 6: Create data.yaml for YOLOv8

import yaml

data_yaml = {
    "path": YOLO_DATASET_DIR,
    "train": "train/images", # Corrected path to match autodistill output
    "val": "valid/images",   # Corrected path to match autodistill output (autodistill uses 'valid')
    "names": [
        "Sharp Object",
        "Choking Hazard",
        "Chemical Danger",
        "Electrical Hazard"
    ]
}

yaml_path = os.path.join(YOLO_DATASET_DIR, "data.yaml")

with open(yaml_path, "w") as f:
    yaml.dump(data_yaml, f)

print("data.yaml created")

In [ ]:
# Block 7: Train YOLOv8 (WORKING VERSION FOR COLAB)

!pip install ultralytics --quiet

import torch
from ultralytics import YOLO
from ultralytics.nn.tasks import DetectionModel
from ultralytics.nn.modules import Conv

# Add necessary classes to PyTorch's safe globals for unpickling
torch.serialization.add_safe_globals(
    [torch.nn.modules.container.ModuleList, DetectionModel, torch.nn.modules.container.Sequential, Conv]
)

# Load pretrained YOLOv8 nano model
model = YOLO("yolov8n.pt")

# Train
model.train(
    data=yaml_path,
    epochs=50,
    imgsz=640,
    batch=8,
    device=0,  # GPU
    project="child_safety_project_runs", # Changed to a simple valid name for wandb integration
    name="yolov8_child_safety"
)

print("Training finished successfully")

In [ ]:
# Block 8: Inference test

import matplotlib.pyplot as plt
import cv2
import os

test_image = os.path.join(RAW_IMAGES_DIR, os.listdir(RAW_IMAGES_DIR)[0])

results = model(test_image, conf=0.3)

# Get the annotated image as a numpy array (BGR format) from the first result
annotated_image_bgr = results[0].plot()

# Convert BGR to RGB for correct display in matplotlib
annotated_image_rgb = cv2.cvtColor(annotated_image_bgr, cv2.COLOR_BGR2RGB)

# Display the image
plt.figure(figsize=(10, 8)) # Optional: Adjust figure size
plt.imshow(annotated_image_rgb)
plt.axis('off') # Hide axes for cleaner display
plt.show()

In [ ]:
# Block 8: Evaluate YOLOv8 model

from ultralytics import YOLO

# Загружаем модель, которую мы только что обучили
# The model is saved to /content/child_safety_project_runs/yolov8_child_safety/weights/best.pt
# instead of PROJECT_ROOT because YOLOv8 saves to a local run directory by default.
model_path = "/content/child_safety_project_runs/yolov8_child_safety/weights/best.pt"
model = YOLO(model_path)

# Оценка модели на валидационном наборе
metrics = model.val(
    data=yaml_path,    # path к data.yaml
    batch=8,
    imgsz=640,
)

# Печатаем основные метрики
print("Evaluation metrics:")
print(f"mAP50: {metrics.box.map50:.4f}")
print(f"mAP50-95: {metrics.box.map:.4f}")
print(f"Precision: {metrics.box.p[0].item():.4f}")
print(f"Recall: {metrics.box.r[0].item():.4f}")